In [31]:
import torch
import torchvision
from mamba_ssm import Mamba
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
from transformers import BertTokenizer
import spacy

In [11]:
nlp = spacy.load('en_core_web_sm')

In [4]:
data = pd.read_csv('data/disaster_tweets/train.csv')

In [5]:
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
data.shape

(7613, 5)

In [12]:
with nlp.disable_pipes():
    text_embeddings = np.array([nlp(text).vector for text in data["text"]])
text_embeddings.shape

(7613, 96)

In [15]:
text_embeddings = text_embeddings.reshape(-1, 96, 1)
text_embeddings.shape

(7613, 96, 1)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(text_embeddings, data["target"], test_size=0.2, random_state=42)

In [23]:
y_train = y_train.to_numpy().reshape(-1, 1)
y_test = y_test.to_numpy().reshape(-1, 1)

In [24]:
y_train.shape

(6090, 1)

In [25]:
train_loader = torch.utils.data.DataLoader(dataset=torch.utils.data.TensorDataset(torch.tensor(X_train), torch.tensor(y_train)), batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=torch.utils.data.TensorDataset(torch.tensor(X_test), torch.tensor(y_test)), batch_size=32, shuffle=True)

In [26]:
model = Mamba(
    # This module uses roughly 3 * expand * d_model^2 parameters
    d_model=1, # Model dimension d_model
    d_state=16,  # SSM state expansion factor
    d_conv=4,    # Local convolution width
    expand=2,    # Block expansion factor
).to("cuda")

In [27]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [29]:
# Function to Train the Model
def train_model(model, train_loader, val_loader, optimizer, device, num_epochs):
    # Loop through the specified number of epochs
    for epoch in range(num_epochs):
        # Set the model to training mode
        model.train()
        # Initialize total loss for the current epoch
        total_loss = 0

        # Loop through the batches in the training data
        for batch in train_loader:
            inputs, labels = [t.to(device) for t in batch]
            
            # Zero the gradients
            optimizer.zero_grad()
            # Perform forward pass
            outputs = model(inputs)
            # Compute the loss
            loss = torch.nn.functional.cross_entropy(outputs, labels)
            # Perform backward pass
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()

        model.eval()  # Set the model to evaluation mode
        val_loss = 0
        # Disable gradient computation during validation
        with torch.no_grad():
            for batch in val_loader:
                inputs, labels = [t.to(device) for t in batch]
                outputs = model(inputs)
                loss = torch.nn.functional.binary_cross_entropy(outputs, labels)
                val_loss += loss.item()
                
        # Print the average loss for the current epoch
        print(
            f'Epoch {epoch+1}, Training Loss: {total_loss/len(train_loader)},Validation loss:{val_loss/len(val_loader)}')


# Call the function to train the model
train_model(model, train_loader, val_loader, optimizer, "cuda", num_epochs=30)

Epoch 1, Training Loss: 4.564115948702028,Validation loss:4.570182998975118
Epoch 2, Training Loss: 4.565161073395095,Validation loss:4.570032795270284
Epoch 3, Training Loss: 4.565259519047762,Validation loss:4.570391496022542
Epoch 4, Training Loss: 4.56471732024747,Validation loss:4.569980452458064
Epoch 5, Training Loss: 4.564972924936504,Validation loss:4.567366222540538
Epoch 6, Training Loss: 4.56478745775073,Validation loss:4.570167571306229
Epoch 7, Training Loss: 4.565071006095846,Validation loss:4.567692776521047
Epoch 8, Training Loss: 4.5644649061232965,Validation loss:4.56870374083519
Epoch 9, Training Loss: 4.564397177771124,Validation loss:4.566272348165512
Epoch 10, Training Loss: 4.5646518487580785,Validation loss:4.567552576462428
Epoch 11, Training Loss: 4.564378106781326,Validation loss:4.5669393340746565
Epoch 12, Training Loss: 4.564309372327715,Validation loss:4.567978203296661
Epoch 13, Training Loss: 4.564571597813312,Validation loss:4.568010479211807
Epoch 14

In [33]:
y_pred = model(torch.tensor(X_test).to("cuda")).argmax(dim=1).cpu().numpy()

In [34]:
f1_score(y_test, y_pred)

0.0